# OPERA CSLC-S1 Data Access Notebook

## 1. Import needed Python packages

In [ ]:
from getpass import getpass
import opensarlab_lib as osl
from pathlib import Path
from pprint import pprint
import re

import asf_search as disco
from asf_search.download.file_download_type import FileDownloadType

## 2. Authenticate with asf_search and start a session

**Gather credentials for authentication with Earth Data Login**

In [ ]:
username = input("Enter your EDL username")
password = getpass("Enter your EDL password")

**Start an asf_search session**

In [ ]:
try:
    user_pass_session = disco.ASFSession().auth_with_creds(username, password)
except disco.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

## 3. Create a directory in which to download the CSLCs

**Create a data directory alongside the Jupyter Book containing this notebook** 

In [ ]:
name = input("Enter the name of a directory to create, which will hold your downloaded CSLCs")
data_dir = Path.cwd().parents[1]/name
data_dir.mkdir(exist_ok=True)
print(f"Data Directory: {data_dir}")

## 4. Search for OPERA CSLC-S1 products

**Option 1: List Search**

- Search by OPERA product ID

- Identify OPERA IDs with an [OPERA-S1 Geographic Search on Vertex](https://search.asf.alaska.edu/#/?maxResults=250&dataset=OPERA-S1)


In [ ]:
product_list = [
    "OPERA_L2_CSLC-S1_T173-370304-IW2_20231018T134413Z_20231019T180049Z_S1A_VV_v1.0",
    "OPERA_L2_CSLC-S1_T173-370304-IW2_20231205T134412Z_20231206T075651Z_S1A_VV_v1.0"
    ]
results = disco.granule_search(product_list)
print(results)

**Option 2: Geographic Search**
- Search by intersection with an area of interest
  - Define in WKT as a POINT or POLYGON
    - `'intersectsWith': 'POINT(-117.4231 35.7597)'`
    - `'intersectsWith': 'POLYGON((-114.9066 35.2997,-114.159 35.2997,-114.159 35.8598,-114.9066 35.8598,-114.9066 35.2997))'`
   
*Full set of available search parameters:*
- *absoluteOrbit: For ALOS, ERS-1, ERS-2, JERS-1, and RADARSAT-1, Sentinel-1A, Sentinel-1B this value corresponds to the orbit count within the orbit cycle. For UAVSAR it is the Flight ID.*
- *asfFrame: This is primarily an ASF / JAXA frame reference. However, some platforms use other conventions. See ‘frame’ for ESA-centric frame searches.*
- *beamMode: The beam mode used to acquire the data.*
- *beamSwath: Encompasses a look angle and beam mode.*
- *campaign: For UAVSAR and AIRSAR data collections only. Search by general location, site description, or data grouping as supplied by flight agency or project.*
- *maxDoppler: Doppler provides an indication of how much the look direction deviates from the ideal perpendicular flight direction acquisition.*
- *minDoppler: Doppler provides an indication of how much the look direction deviates from the ideal perpendicular flight direction acquisition.*
- *end: End date of data acquisition. Supports timestamps as well as natural language such as "3 weeks ago"*
- *maxFaradayRotation: Rotation of the polarization plane of the radar signal impacts imagery, as HH and HV signals become mixed.*
- *minFaradayRotation: Rotation of the polarization plane of the radar signal impacts imagery, as HH and HV signals become mixed.*
- *flightDirection: Satellite orbit direction during data acquisition*
- *flightLine: Specify a flightline for UAVSAR or AIRSAR.*
- *frame: ESA-referenced frames are offered to give users a universal framing convention. Each ESA frame has a corresponding ASF frame assigned. See also: asfframe*
- *granule_list: List of specific granules. Search results may include several products per granule name.*
- *groupID: Identifier used to find products considered to be of the same scene but having different granule names*
- *insarStackId: Identifier used to find products of the same InSAR stack*
- *instrument: The instrument used to acquire the data. See also: platform*
- *intersectsWith: Search by polygon, linestring, or point defined in 2D Well-Known Text (WKT)*
- *lookDirection: Left or right look direction during data acquisition*
- *offNadirAngle: Off-nadir angles for ALOS PALSAR*
- *platform: Remote sensing platform that acquired the data. Platforms that work together, such as Sentinel-1A/1B and ERS-1/2 have multi-platform aliases available. See also: instrument*
- *polarization: A property of SAR electromagnetic waves that can be used to extract meaningful information about surface properties of the earth.*
- *processingDate: Used to find data that has been processed at ASF since a given time and date. Supports timestamps as well as natural language such as "3 weeks ago"*
- *processingLevel: Level to which the data has been processed*
- *product_list: List of specific products. Guaranteed to be at most one product per product name.*
- *relativeOrbit: Path or track of satellite during data acquisition. For UAVSAR it is the Line ID.*
- *season: Start and end day of year for desired seasonal range. This option is used in conjunction with start/end to specify a seasonal range within an overall date range.*
- *start: Start date of data acquisition. Supports timestamps as well as natural language such as "3 weeks ago"*
- *collections: List of collections (concept-ids) to limit search to*
- *temporalBaselineDays: List of temporal baselines, used for Sentinel-1 Interferogram (BETA)*
- *maxResults: The maximum number of results to be returned by the search*
- *opts: An ASFSearchOptions object describing the search parameters to be used. Search parameters specified outside this object will override in event of a conflict.*

In [ ]:
options = {
    'intersectsWith': 'POINT(-121.643 38.127)',
    'dataset': 'OPERA-S1',
    'start': '2019-07-03T00:00:00Z',
    'end': '2024-01-31T00:00:00Z',
    'processingLevel': [
        'CSLC'
    ],
    'maxResults': '1000'
}

results = disco.search(**options)
burst_ids = list(set([r.properties['operaBurstID'] for r in results]))
print(f"Found {len(results)} RTCs with {len(burst_ids)} burst IDs")

## 5. Filter the Results

**Select burst IDs to download**

In [ ]:
print("\nSelect the burst IDs you wish to download")
burst_select = osl.select_mult_parameters(burst_ids)
display(burst_select)

In [ ]:
bursts = burst_select.value
selected_results = [r for r in results if r.properties['operaBurstID'] in bursts]
selected_results

**When >1 CSLC are found for a burst-pair, select the most recently processed CSLC**

The geographic search may return current and superceded data


In [ ]:
filtered_bursts = dict()
acquisition_date_regex = r"(?<=OPERA_L2_CSLC-S1_)T\d{3}-\d{6}-IW\d_\d{8}T\d{6}Z(?=_\d{8}T\d{6}Z)"
process_date_regex = '(?<=OPERA_L2_CSLC-S1_T\d{3}-\d{6}-IW\d_\d{8}T\d{6}Z_)\d{8}T\d{6}Z'

for b in selected_results:
    filename = b.properties['fileName']
    try:
        id_date = re.search(acquisition_date_regex, filename).group(0)
        try:
            # for scenes that only differ by processing date, we can use a simple relational comparisons
            if filtered_bursts[id_date].properties['fileName'] < filename:
                filtered_bursts[id_date] = b
        except KeyError:
            filtered_bursts[id_date] = b
    except AttributeError:
        raise Exception(f"Acquisition not found in filename: {str(b)}")
        
filtered_bursts = list(filtered_bursts.values())   

for b in filtered_bursts:
    print(b.properties['fileName'])

## 6. Download Data

**Download CSLCs**

In [ ]:
for b in filtered_bursts:
    cslc_dir = data_dir/b.properties['operaBurstID']/b.properties['fileID']
    print(cslc_dir)
    cslc_dir.mkdir(exist_ok=True, parents=True)
    b.download(cslc_dir, session=user_pass_session, fileType=FileDownloadType.ALL_FILES)

**Download the static layers**

In [ ]:
for b in bursts:
    options = {
    'dataset': 'OPERA-S1',
    'operaBurstID': b,
    'processingLevel': ['CSLC-STATIC'],
    'maxResults': '1000'
    }
    results = disco.search(**options)
    if len(results) > 0:
        static_dir = data_dir/b/f"{b}_STATIC"
        static_dir.mkdir(exist_ok=True)
        results[0].download(static_dir, 
                            session=user_pass_session, 
                            fileType=FileDownloadType.ALL_FILES)
    